In [ ]:
import requests, pandas, getpass

api = 'https://api.earthref.org/v1/MagIC/{}'
handle = input()
password = getpass.getpass()

### Create a Private Contribution and delete it
If no errors are reported, this cell can be repeated without any side effects. If noone has also made a private contribution between repeated executions, the same contribution ID should be reused.

In [ ]:
create_response = requests.post(api.format('private'), auth=(handle, password))
print(create_response.request.method, create_response.request.url)
if (create_response.status_code == 200):
    contribution_id = create_response.json()['id']
    print('Created private contribution with ID', contribution_id, '\n')

    delete_response = requests.delete(api.format('private'), params={'id': contribution_id}, auth=(handle, password))
    print(delete_response.request.method, delete_response.request.url)
    if (delete_response.status_code == 200):
        print('Deleted private contribution with ID', contribution_id, '\n')
    else:
        print('Delete Private Contribution Error:', delete_response.json()['errors'][0]['message'], '\n')
else:
    print('Create Private Contribution Error:', create_response.json()['errors'][0]['message'], '\n')

### Download a Public Contribution and create a Private Contribution to upload it to

In [ ]:
contribution_id = 16901

response = requests.get(api.format('data'), params={'id': contribution_id})
print(response.request.method, response.request.url)
if (response.status_code == 200):
    contribution_file = 'downloads/magic_contribution_{}'.format(contribution_id)
    open(contribution_file, 'w').write(response.text)
    print('Retrieved contribution data with ID', contribution_id, '\n')
    
    create_response = requests.post(api.format('private'), auth=(handle, password))
    print(create_response.request.method, create_response.request.url)
    if (create_response.status_code == 200):
        new_contribution_id = create_response.json()['id']
        print('Created private contribution with ID', new_contribution_id, '\n')
        
        with open(contribution_file, 'rb') as f:
            upload_response = requests.put(api.format('private'), 
                params={'id': new_contribution_id}, 
                auth=(handle, password), 
                headers={'Content-Type': 'text/plain'}, 
                data=f
            )

        print(upload_response.request.method, upload_response.request.url)
        if (upload_response.status_code == 200):
            print('Uploaded a text file to private contribution with ID', contribution_id, '\n')
        else:
            print('Upload Private Contribution Error:', upload_response.json()['errors'][0]['message'], '\n')
    else:
        print('Create Private Contribution Error:', create_response.json()['errors'][0]['message'], '\n')
else:
    print('Retrieve Public Contribution Error:', response.json()['errors'][0]['message'], '\n')